<a href="https://colab.research.google.com/github/Crazy-Explorer31/News-assistant/blob/main/experiments/gradient_boosting_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%pip install --upgrade scipy


In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import sparse
import requests

In [9]:
tfidf_train = sparse.load_npz("tfidf_train_sparse.npz")

In [10]:
tfidf_test = sparse.load_npz("tfidf_test_sparse.npz")

In [11]:
datasets = {

  "y_test.csv" : "https://www.dropbox.com/scl/fi/3c7br6x2k0lo6iujtwzt0/y_test.csv?rlkey=9rlywjvoduahpu27uoclmmtqg&st=5hyfuf7b&dl=1",

  "y_train.csv" : "https://www.dropbox.com/scl/fi/n228uag3g0qqdigwlu151/y_train.csv?rlkey=8g2nm1w539t7o84k8jceqfdtz&st=tskbwl6g&dl=1",

  "X_train.csv" : "https://www.dropbox.com/scl/fi/eraukmu5w1r56vigj1bla/X_train.csv?rlkey=gpzrwnvhhk6q07r9nklz09fl1&st=vnux3ves&dl=1",

  "X_test" : "https://www.dropbox.com/scl/fi/a4dlgfke5swfjkolikr5o/X_test.csv?rlkey=2u02d3zmy0ls59katzrwljpgq&st=w6w9egec&dl=1"
}

def get_dataset(url, dataset_name):
    response = requests.get(url)

    with open(dataset_name, 'wb') as file:
        file.write(response.content)

    df = pd.read_csv(dataset_name)
    return df


In [12]:
datasets

{'y_test.csv': 'https://www.dropbox.com/scl/fi/3c7br6x2k0lo6iujtwzt0/y_test.csv?rlkey=9rlywjvoduahpu27uoclmmtqg&st=5hyfuf7b&dl=1',
 'y_train.csv': 'https://www.dropbox.com/scl/fi/n228uag3g0qqdigwlu151/y_train.csv?rlkey=8g2nm1w539t7o84k8jceqfdtz&st=tskbwl6g&dl=1',
 'X_train.csv': 'https://www.dropbox.com/scl/fi/eraukmu5w1r56vigj1bla/X_train.csv?rlkey=gpzrwnvhhk6q07r9nklz09fl1&st=vnux3ves&dl=1',
 'X_test': 'https://www.dropbox.com/scl/fi/a4dlgfke5swfjkolikr5o/X_test.csv?rlkey=2u02d3zmy0ls59katzrwljpgq&st=w6w9egec&dl=1'}

In [30]:
y_train = get_dataset(datasets['y_train.csv'], 'y_train.csv')['category']
y_test = get_dataset(datasets['y_test.csv'], 'y_test.csv')['category']

In [31]:
y_test

,category
0,1
1,2
2,0
3,0
4,4
...,...
86665,0
86666,2
86667,2
86668,7


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(max_iter=500, random_state=42)

log_reg.fit(tfidf_train, y_train)

y_pred = log_reg.predict(tfidf_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность на тестовой выборке:", accuracy)

Точность на тестовой выборке: 0.9454020999192339


In [33]:
y_pred_train = log_reg.predict(tfidf_train)
accuracy = accuracy_score(y_train, y_pred_train)
print("Точность на тренировочной выборке:", accuracy)

Точность на тренировочной выборке: 0.9620030955312594


In [37]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

starndart_bossting = GradientBoostingClassifier(learning_rate=0.5,
                                                n_estimators=10,
                                                min_samples_split=2,
                                                min_samples_leaf=1,
                                                max_depth=3,
                                                random_state=42,
                                                verbose=1)

starndart_bossting.fit(tfidf_train, y_train)


      Iter       Train Loss   Remaining Time 
         1           1.0060           17.97m
         2           0.7248           15.96m
         3           0.6008           13.93m
         4           0.5280           11.90m
         5           0.4764            9.87m
         6           0.4391            7.88m
         7           0.4092            5.90m
         8           0.3848            3.92m
         9           0.3625            1.96m
        10           0.3460            0.00s


GradientBoostingClassifier(learning_rate=0.5, n_estimators=10, random_state=42,
                           verbose=1)

In [38]:
y_pred = starndart_bossting.predict(tfidf_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность на тестовой выборке:", accuracy)

y_pred_train = starndart_bossting.predict(tfidf_train)
accuracy = accuracy_score(y_train, y_pred_train)
print("Точность на тренировочной выборке:", accuracy)

Точность на тестовой выборке: 0.8954655590169609
Точность на тренировочной выборке: 0.8990243637100882


GradientBoostingClassifier обучается долго и дает слабую точность

In [39]:
import lightgbm as lgb

lgb_boosting = lgb.LGBMClassifier(learning_rate=0.1,
                                  n_estimators=100,
                                  max_depth=4,
                                  random_state=42,
                                  n_jobs=10,
                                  verbose=-1)

lgb_boosting.fit(tfidf_train, y_train)

y_pred = lgb_boosting.predict(tfidf_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность на тестовой выборке:", accuracy)

y_pred_train = lgb_boosting.predict(tfidf_train)
accuracy = accuracy_score(y_train, y_pred_train)
print("Точность на тренировочной выборке:", accuracy)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Точность на тестовой выборке: 0.9300103842159917


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Точность на тренировочной выборке: 0.9373080745894465


lgb.LGBMClassifier дает точность повыше, но все еще хуже, чем простая гогистическая регрессия